In [1]:
import numpy as np
import matplotlib.pyplot as plt

from preprocessing.preprocessor import Preprocess     #  From usatellite
from models.unet2d.unet2d_model import Unet2d

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from skimage.transform import resize

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from tensorflow.keras import backend as K

from tensorflow.keras.optimizers import Adam
from loss.loss_metrics import Loss

In [2]:
imgs_array = np.load('/Users/cody/Python/usattelite_data/img_array.npy')

In [3]:
dsm_array = np.load('/Users/cody/Python/usattelite_data/dsm_array.npy')

In [4]:
nvdi_array = np.load('/Users/cody/Python/usattelite_data/veggies_array.npy')

In [5]:
label_array = np.load('/Users/cody/Python/usattelite_data/label_array.npy')

Let's just start with our images as input to the network to provide a benchmark...  Later we will create more complex data structures, which contain more than just rgb.  These data will add the dsm and/or nvdi images as extra channels.

First we will scale our data, since that is typically helpful for machine learning.

In [6]:
preproc = Preprocess()

In [7]:
imgs_array_scaled = preproc.unit_normalize_dims(imgs_array)

I'm actually going to just use sklearn's train test split to split up the data into traing and test data - there's really no reason to write our own code for this.  We will also shuffle the data and set a random state so it's reproducible.

In [8]:
x_train, x_val, y_train, y_val = train_test_split(imgs_array, label_array, test_size=0.30, shuffle=True, random_state=42)

In [9]:
unet2dmodel = Unet2d(input_img=imgs_array)

In [10]:
unet2dmodel_obj = unet2dmodel.get_unet()

In [11]:
unet2dmodel_obj.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=["categorical_accuracy", Loss.f1])

In [ ]:
results = unet2dmodel_obj.fit(x_train, y_train, batch_size=1, epochs=100, callbacks=unet2dmodel.callback_list(),
                   validation_data=(x_val, y_val))

Epoch 1/100
 4/23 [====>.........................] - ETA: 2:13 - loss: 1.7986 - categorical_accuracy: 0.1999 - f1: 0.2774